## Install

``` shell
conda create python=2.7 h5py jupyter -p ~/space/envs/peaknet
conda install pytorch=0.1.12 torchvision cuda80 -c pytorch
conda install tensorboardX
```


### Import packages

In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torchvision
import h5py
import numpy as np
from peaknet import Peaknet

/reg/neh/home/liponan/space/envs/peaknet/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Load data from CXI

In [2]:
def load_from_cxi( filename, idx, box_size=7 ):
    f = h5py.File(filename, 'r')
    nPeaks = f["entry_1/result_1/nPeaks"].value
    dataset_hits = len(nPeaks)
    print('hits: ' + str(dataset_hits))
    dataset_peaks = np.sum(nPeaks)
    print('peaks: ' + str(dataset_peaks))
    img = f["entry_1/data_1/data"][idx,:,:]
    mask = f["entry_1/data_1/mask"][idx,:,:]
    img = img * (1-mask)
    x_label = f['entry_1/result_1/peakXPosRaw'][idx,:]
    y_label = f['entry_1/result_1/peakYPosRaw'][idx,:]
    f.close()

    imgs = np.reshape( img, (8, 185, 4, 194*2) )
    imgs = np.transpose( imgs, (0, 2, 1, 3) )
    imgs = np.reshape( imgs, (1, 32, 185, 388) )
    n, m, h, w = imgs.shape

    cls = np.zeros( (nPeaks[idx],) )
    s = np.zeros( (nPeaks[idx],) )
    r = np.zeros( (nPeaks[idx],) )
    c = np.zeros( (nPeaks[idx],) )
    ww = np.zeros( (nPeaks[idx],) )
    hh = np.zeros( (nPeaks[idx],) )
    for u in range(nPeaks[idx]):
        my_s = (int(y_label[u])/185)*4 + (int(x_label[u])/388)
        my_r = y_label[u] % 185
        my_c = x_label[u] % 388
        s[u] = my_s
        r[u] = my_r
        c[u] = my_c
        hh[u] = box_size
        ww[u] = box_size
    labels = (cls, s, r, c, hh, ww)

    return imgs, labels

In [3]:
filename = "/reg/neh/home/liponan/data/cxic0415/r0091/cxic0415_0091.cxi.backup"
idx=468
imgs, labels = imgs, labels = load_from_cxi( filename, idx )
print("imgs", imgs.shape)
print("labels", len(labels), len(labels[0]))

hits: 1325
peaks: 81069
('imgs', (1, 32, 185, 388))
('labels', 6, 454)


### Init. network

In [16]:
net = Peaknet()
net.loadCfg( "../pytorch-yolo2/cfg/newpeaksv10-asic.cfg" )
# net.init_model()
net.loadWeights( "../pytorch-yolo2/cfg/newpeaksv10-asic.cfg", 
                "../darknet/backup/newpeaksv10_100.weights")
net.model.cuda()

Darknet (
  (models): ModuleList (
    (0): Sequential (
      (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
      (leaky1): LeakyReLU (0.1, inplace)
    )
    (1): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (2): Sequential (
      (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (leaky2): LeakyReLU (0.1, inplace)
    )
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (4): Sequential (
      (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
      (leaky3): LeakyReLU (0.1, inplace)
    )
    (5): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (6): Sequential (
      (conv4): Conv2d(128, 256, kernel_size=(1, 1), stride=(

### Predict

In [ ]:
results = net.predict( imgs, batch_size=32, use_cuda=True )

In [ ]:
torch.cuda.is_available()